## OSRM (Open Source Routing Machine)

OSRM is an open-source software project that provides high-performance routing services.

## OSRM Tutorial on Ubuntu 20.04 and 22.04

This tutorial provides instructions and examples on how to install, configure, and utilise OSRM for calculating routes and performing various routing-related tasks.

## Prerequisites: (execute in command line)

This is a very important part of the tutorial and the osrm project. 

The osrm software seems to work better with a fresh installation of Linux distribution because there are fewer conflicts with the following prerequisites. 

The installation has been verified on several Ubuntu machines and must work with the prerequisites below:

```bash
$ sudo apt update
```
```bash
$ sudo apt install build-essential git cmake screen pkg-config doxygen \
libboost-all-dev libtbb-dev lua5.2 liblua5.2-dev libluabind-dev \
libstxxl-dev libstxxl1v5 libxml2 libxml2-dev libosmpbf-dev \
libbz2-dev libzip-dev libprotobuf-dev
```

## Install OSRM Backend: (execute in jupyter notebook)

In [ ]:
%cd /home/mghorbani/workspace/osrm-tutorial

In [ ]:
!git clone https://github.com/Project-OSRM/osrm-backend.git

In [ ]:
import os
os.environ['CXXFLAGS'] = "-DBOOST_ALLOW_DEPRECATED_HEADERS -DBOOST_BIND_GLOBAL_PLACEHOLDERS"

In [ ]:
# Create the directory if it doesn't exist
os.makedirs('osrm-install', exist_ok=True)

# Change the current working directory
%cd osrm-install/

In [ ]:
# Create the directory if it doesn't exist
os.makedirs('build', exist_ok=True)

# Change the current working directory
%cd build/

In [ ]:
!cmake -DCMAKE_INSTALL_PREFIX="../../osrm-install" ..
!make -j 4 >cmake_output.log 2>&1
!make install >>cmake_output.log 2>&1

In [ ]:
# Adding OSRM executables path to the system path
os.environ["PATH"] = "../../osrm-install/bin:" + os.environ["PATH"]

## Known Issues During OSRM-backend Installation:

This issue can be experienced in the Linux Ubuntu20.04 system during the make process: 

```bash
undefined reference to `lua_newuserdatauv'
collect2: error: ld returned 1 exit status
```
If this occurs, please restart the installation process by specifying the Lua5.2 directory as:

```bash
$ cd osrm-backend/
$ mkdir -p build/
$ cd build/
$ cmake -DCMAKE_INSTALL_PREFIX="../../osrm-install" -DLUA_INCLUDE_DIR="/usr/include/lua5.2" ..
$ make -j 4
$ make install
```

If similar issues occur for any other libraries (e.g., TBB, Boost and Doxygen), specify installation prefixes to be searched by cmake using `-DCMAKE_PREFIX_PATH="paths"`. Separate the paths by `;` semicolon.

## Brief Description of Installed OSRM Executables:

To use the OSRM executables, it is generally recommended to follow the sequence of steps outlined below:

**osrm-extract:**<br> 
This executable is used to extract the street network data from OpenStreetMap (OSM) data and convert it into the OSRM-specific format. You typically provide the OSM data file as input and specify the desired options, such as the profile to use and any additional restrictions or customizations.

**Example command:**<br>
osrm-extract map.osm.pbf -p profile.lua
The output of this step is a preprocessed .osrm file containing the street network data in a format optimized for routing.

**osrm-partition:**<br>
osrm-partition: This executable performs a spatial partitioning of the preprocessed street network data to enable efficient routing queries. It splits the data into smaller regions, which improves query performance. This step is optional but recommended for larger datasets.

**Example command:**<br>
osrm-partition map.osrm
This step generates a .osrm.partition file that stores the partitioned data.

**osrm-customize:**<br>
This executable further optimizes the routing data to improve query performance. It modifies the preprocessed data based on heuristics and optimization techniques specific to OSRM.

**Example command:**<br>
osrm-customize map.osrm
The output of this step is a final .osrm file that is ready for routing queries.

**osrm-routed:**<br> 
This executable starts the OSRM server, allowing you to make routing queries via HTTP requests or use the OSRM API. It loads the final .osrm file and provides a server interface to handle routing requests.

**Example command:**<br>
osrm-routed map.osrm
After executing this command, the OSRM server will start and listen for routing queries on a specified port (default is 5000).

**osrm-contract:**<br>
osrm-contract command should be executed after the osrm-extract step and before the osrm-routed step. It takes the preprocessed .osrm file as input and generates a contracted .osrm file.

**Example command:**<br>
osrm-contract map.osrm
The contracted .osrm file produced by osrm-contract is used in the subsequent osrm-routed step to start the OSRM server and handle routing queries.

## Retrieving OSRM Routing Data:

The initial OpenStreetMap (osm) map data must be downloaded (into the osrm-backend directory, possibly) from https://planet.openstreetmap.org/pbf/ in pbf.  

PBF stands for Protocolbuffer Binary Format. It is a binary data format used for efficiently storing and exchanging structured data. PBF is commonly used in the context of geographic data, particularly for storing and distributing OSM data.

Other available sources:

http://download.geofabrik.de

https://download.bbbike.org/osm/

Note: The larger the geographical location is, the bigger the pbf file would be. 

The largest available is the map of planet exists in pbf here:

http://planet.openstreetmap.org/pbf/planet-latest.osm.pbf 

with almost **60G** of data.

## Get .pbf data

You can also use the command below to download a file to osrm-backend/ directory: (disk space warning) 

```bash
$ wget -c http://planet.openstreetmap.org/pbf/planet-latest.osm.pbf -P /path/to/osrm-backend
```

For a map of an individual country, state, province or city, go to http://download.geofabrik.de and download the map. In this instance, Great Britain with almost **1.5G** with the following command:

```bash
$ wget -c http://download.geofabrik.de/europe/great-britain-latest.osm.pbf -P /path/to/osrm-backend
```

Note: Please make sure to replace "/path/to/" with the actual path of osrm-backend/ or use `wget -c` in osrm-beckend/ without the `-P /path/to/osrm-backend` option.

Following the download, great-britain-latest.osm.pbf will be present in osrm-backend directory.

## Issues with large osm.pbf files:

Although the large pbf files can be downloaded on your system, the extraction may fail due to the system memory space. Therefore, it is recommended to download smaller regions and then merge them with the osmium utility.

First download a region (e.g., Mali) and all required neighbouring regions (e.g., Nigeria, Niger, Burkina Faso etc.). Each download provides a pbf file.

In Ubuntu, install the utility by:

```bash
$ sudo apt install osmium-tool
```

And then, merge all regions into a new merge.osm.pbf file:

```bash
$ osmium merge *.osm.pbf -o merged.osm.pbf
```

If the merged file becomes too large, you may consider regenerating a smaller one. 


## Generating OSRM Routing Data:

If you have added the path to OSRM executables to your system PATH (export PATH="/path/to/osrm-install/bin:$PATH") successfully, running the command below:

```bash
$ osrm-`double-tab`
```
    
Should return:

osrm-components<br>
osrm-contract<br>
osrm-customize<br>
osrm-datastore<br>
osrm-extract<br>
osrm-partition<br>
osrm-routed
    
You can now execute the command below in osrm-backend/ directory to extract the osm.pbf file:

```bash
$ osrm-extract great-britain-latest.osm.pbf --threads=10
```

Note that executing this command might take a long time even with more threads. Consider using `screen` to avoid interruptions during the process.

## Running commands in screen

`screen` is a terminal multiplexer that allows you to create multiple virtual terminal sessions within a single terminal window or SSH session. It provides a way to run commands and applications that need to keep running even after you disconnect from the terminal or SSH session.

Some useful commands in `screen`:

screen: Start a new screen session.<br>
Ctrl + A, C: Create a new window within the screen session.<br>
Ctrl + A, N: Switch to the next window.<br>
Ctrl + A, P: Switch to the previous window.<br>
Ctrl + A, ": Show a list of all windows for easy selection.<br>
Ctrl + A, A: Rename the current window.<br>
Ctrl + A, D: Detach from the screen session (leave it running in the background).

screen -r: Reattach to a detached screen session.<br>
screen -ls: List all active screen sessions.

exit or Ctrl + D: Close a terminal session inside a screen window.

Screen provides many more features and commands. You can refer to the screen documentation or use the man screen command for more detailed information.

Here's an example of running the osrm-extract command in screen:

1. In osrm-backend directory, start a new screen session with name osrm. 

```bash
$ screen -S osrm
```

2. Run the osrm-extract command within the screen session:

```bash
$ osrm-extract great-britain-latest.osm.pbf --threads=10
```

3. Detach from the screen session by pressing Ctrl + A, then D. This leaves the process running in the background.

To reattach to the screen session later, use the following command:

```bash
$ screen -r osrm
```

This will bring you back to the osrm screen session and allow you to check the progress or interact with the osrm-extract process.

To list active screen sessions, if any, along with their session IDs and status, run:

```bash
$ screen -ls
```

For example, the output may look like this:

```bash
There is a screen on:
        22478.osrm      (06/13/23 18:45:12)     (Detached)
1 Socket in /run/screen/S-mghorbani.
```

Following the execution of osrm-extract, many extracted files will appear in osrm-backend/ directory.

## Additioanl Information

By default, lua.car (script) is used during the OSRM data processing stage. The script is used by the osrm-extract command to extract relevant data from OpenStreetMap files and generate a custom routing graph. 

The Lua script defines the routing profile, including the road network attributes, vehicle restrictions, speed limits, and other factors that influence the routing decisions made by OSRM. The generated routing graph is then used by other OSRM components, such as osrm-routed, to provide routing services based on the defined profile.

The lua.car or a custom profile (profile.lua below) contains Lua script defining the routing profile with specific attributes and restrictions like:

```lua
-- profile.lua script:

-- Configuration for car routing

-- Define the vehicle profile
local profile = {
  default_speed = 30,  -- Default speed in kilometers per hour
  highway_speeds = {
    motorway = 100,  -- Speed on motorways
    primary = 70,    -- Speed on primary roads
    secondary = 50,  -- Speed on secondary roads
    tertiary = 40    -- Speed on tertiary roads
  },
  access = {  -- Access restrictions
    ["private"] = false,  -- Private roads not accessible
    ferry = true          -- Ferries are accessible
  },
  turn_costs = {  -- Turn costs in seconds
    default = 0,     -- Default turn cost
    ["turn"] = 5,    -- Cost for normal turns
    ["sharp_turn"] = 10,  -- Cost for sharp turns
    ["uturn"] = 20        -- Cost for U-turns
  }
}

-- Function to apply the profile
function process_way(profile, way, result)
  -- Set the speed based on the highway type
  local highway = way:get_value_by_key("highway")
  local speed = profile.highway_speeds[highway] or profile.default_speed
  result.forward_speed = speed
  result.backward_speed = speed

  -- Set the access restriction
  local access = way:get_value_by_key("access")
  if profile.access[access] ~= nil then
    result.forward_access = profile.access[access]
    result.backward_access = profile.access[access]
  end
end

-- Apply the profile to ways
way_handler = process_way

-- Return the profile
return profile
```

A completed script can be added to osrm-extract execution, optionally by:

```bash
$ osrm-extract great-britain-latest.osm.pbf -p profile.lua --threads=10
```

## Partition Map Recursively into Cells:

You can either run this command in a screen session or in the current terminal:

```bash
$ osrm-partition great-britain-latest.osrm
```

Note that this command can take a long time to complete.

## Customize Map Cells:

You can customise the cells by calculating routing weights for all cells by command:

```bash
$ osrm-customize great-britain-latest.osrm
```

This command also may take a long time to complete.

## Run OSRM Engine:

To ensure the continuous operation of the OSRM engine, it is recommended to start it within a screen session, which allows the process to run in the background even when you disconnect from the terminal. This way, you can start the OSRM engine, detach from the screen session, and leave it running while you continue with other tasks. 

Later, you can reattach to the screen session to monitor or interact with the running OSRM engine. This ensures that the engine remains active and accessible even after you close the terminal or log out of the system.

You can start the routing engine by the command:

```bash
$ osrm-routed --algorithm=MLD great-britain-latest.osrm
```

Here are some commonly used routing algorithms in OSRM:

**Dijkstra:**<br> 
The classic Dijkstra algorithm for finding the shortest path in a graph. It considers all edges and vertices to determine the optimal route.

**CH (Contraction Hierarchies):**<br>
A speed-up technique that pre-processes the road network by contracting nodes based on their importance. CH algorithms can significantly reduce the search space and improve route calculation performance.

**MLD (Multi-Level Dijkstra):**<br> 
As mentioned before, MLD is a hierarchy-based algorithm that breaks down the road network into levels of detail. It allows for faster route calculations by considering different levels based on the proximity of the source and destination.

**CoreCH (Core Contraction Hierarchies):**<br>
An advanced CH variant that takes advantage of multi-core processors to further speed up route calculations.

**MLDT (Multi-Level Timed Dijkstra):**<br> 
An extension of MLD that incorporates time-dependent travel information. It considers temporal aspects such as traffic conditions to provide accurate time-based route calculations.

Please visit https://project-osrm.org/ for more inforamtion.

## Check OSRM Engine Running:

Following the execution of the osrm-routed command, you will see a similar output like:

```bash
[2023-06-14T02:38:40.669969316] [info] starting up engines, v5.28.0
[2023-06-14T02:38:40.670057607] [info] Threads: 12
[2023-06-14T02:38:40.670066382] [info] IP address: 0.0.0.0
[2023-06-14T02:38:40.670068982] [info] IP port: 5000
[2023-06-14T02:38:44.455122040] [info] http 1.1 compression handled by zlib version 1.2.11
[2023-06-14T02:38:44.455637165] [info] Listening on: 0.0.0.0:5000
[2023-06-14T02:38:44.455821006] [info] running and waiting for requests
```

This indicates that, OSRM engine has started up and it is ready to serve requests by Listening on IP-Address:Port (0.0.0.0:5000). The address can be localhost:5000 too.

You can also check if the OSRM engine is running by using the `ps` command in Linux:

```bash
$ ps aux | grep osrm-routed
```

Look for the process in the output. If you see an entry with "osrm-routed" and the corresponding details, it means the OSRM engine is running.

## Request Assembly:

After running osrm-routed and having the OSRM engine up and running, you can make requests to the engine to obtain routing information. The requests typically include the source and destination coordinates or waypoints, and the engine responds with the corresponding route information. You can specify additional parameters like the routing algorithm, coordinates precision, and other options.

A few examples of request assembly:

**Walking route:**<br>
origin = "longitude1,latitude1"<br>
destination = "longitude2,latitude2"<br>
request_url = f"http://localhost:5000/route/v1/walking/{origin};{destination}"

**Cycling route:**<br>
origin = "longitude1,latitude1"<br>
destination = "longitude2,latitude2"<br>
request_url = f"http://localhost:5000/route/v1/cycling/{origin};{destination}"

**Route with waypoints:**<br>
origin = "longitude1,latitude1"<br>
waypoint1 = "longitude2,latitude2"<br>
waypoint2 = "longitude3,latitude3"<br>
destination = "longitude4,latitude4"<br>
request_url = f"http://localhost:5000/route/v1/driving/{origin};{waypoint1};{waypoint2};{destination}"

**Route with alternative routes:**<br>
origin = "longitude1,latitude1"<br>
destination = "longitude2,latitude2"<br>
request_url = f"http://localhost:5000/route/v1/driving/{origin};{destination}?alternatives=true"


## Making Requests and Handling Output:

`curl` is a command-line tool for making HTTP requests. 

Here's how you can use `curl` to make a request:

**Basic GET request:**<br>
curl "http://localhost:5000/route/v1/driving/longitude1,latitude1;longitude2,latitude2"

**GET request with query parameters:**<br>
curl "http://localhost:5000/route/v1/driving/longitude1,latitude1;longitude2,latitude2?steps=true&alternatives=true"

**POST request with JSON data:**<br>
curl -X POST -H "Content-Type: application/json" -d '{"coordinates": [[longitude1, latitude1], [longitude2, latitude2]]}' http://localhost:5000/route/v1/driving

**POST request with form data:**<br>
curl -X POST -d "coordinates=longitude1,latitude1;longitude2,latitude2" http://localhost:5000/route/v1/driving

**Adding headers to the request:**<br>
curl -H "Authorization: Bearer token123" http://localhost:5000/route/v1/driving/longitude1,latitude1;longitude2,latitude2


The output of a request can be in various formats such as JSON, GeoJSON, GPX, or text. The response contains details about the route, including the distance, duration, geometry, and turn-by-turn instructions. You can process and utilise this information for further analysis, visualisation, or integration into your own applications.

To make a request to the OSRM engine, you can use HTTP client libraries, tools like cURL, or directly send HTTP requests programmatically. The engine will process the request and provide the routing response in the chosen output format.

A response in JSON format contains information about the route. 

Here's a breakdown of an example response structure:

"code": "Ok": Indicates that the request was processed successfully.<br>
"routes": An array of routes that were found. In this case, there is a single route in the array.<br>
"geometry": The encoded geometry of the route. This represents the path of the route on the map.<br>
"legs": An array of legs that make up the route. In this case, there is a single leg in the array.<br>
"steps": An array of steps within the leg. Since there are no steps in this case, the array is empty.<br>
"summary", "weight", "duration", "distance": These properties provide summary information about the route.<br>
"waypoints": Each waypoint includes information such as the "hint", "distance", "name", and "location".

Here's a simple bash script that makes a request to the OSRM engine using the provided location details, and stores output to "routes.geojson" file:

```shell
#!/bin/bash

# Location details
name1='London'
lat1=51.50837083467904
lon1=-0.13187578769508834
name2='Birmingham'
lat2=52.498968318134516
lon2=-1.8030022360693643

# Prepare API request URL
api_url="http://localhost:5000/route/v1/driving/${lon1},${lat1};${lon2},${lat2}?alternatives=true&steps=true&geometries=polyline&overview=full"

# Send request and save response
response=$(curl -s "$api_url")

# Save response as GeoJSON only if the file is empty
if [ ! -s route.geojson ]; then
    echo "$response" > route.geojson
fi

echo "Route from $name1 to $name2 is saved as route.geojson"
```

**We have added a copy of osrm-request.sh in the parent directory of the project.**

The script can be run using Python in the cell below:

In [ ]:
import subprocess
import os

# Set the file name of the bash script
bash_script = 'osrm-request.sh'

# Set the path to the where osrm-request.sh is stored
bash_script_path = '.'

# Check if the bash script exists
if not os.path.exists(os.path.join(bash_script_path, bash_script)):
    print(f"Error: {bash_script} script not found.")
    exit(1)

# Set the command to change the permission of the bash script
change_permission_cmd = f'chmod +x {os.path.join(bash_script_path, bash_script)}'

# Set the command to execute the bash script
execute_script_cmd = f'./{bash_script}'

try:
    # Change the permission of the bash script
    subprocess.run(change_permission_cmd, shell=True, cwd=bash_script_path)

    # Execute the bash script
    subprocess.run(execute_script_cmd, shell=True, cwd=bash_script_path)
except subprocess.SubprocessError as e:
    print(f"Error executing {bash_script}: {str(e)}")

You can check the content of generated route.geojson in osrm-tutorial/ directory by:

```bash
$ less route.geojson
```

Scroll down to inspect the data.

or, run the cell below to print the first 10 lines of the route.geojson file in here:

In [ ]:
!head -n 10 route.geojson

## Visualising Data Using Maps:

Once the route.geojson is created, the data can be visualised using Folium or gmaps.

The script below is designed to read data from a geojson file and show it on a map, however, before running the script, let's install a few packages that we need for the tasks ahead.

Install packages in the current Jupyter kernel using the cell below:

In [ ]:
import sys
!{sys.executable} -m pip install pandas folium polyline

This code below essentially reads the GeoJSON data, extracts the necessary information (coordinates, geometry), and uses Folium to create a map visualising the route with markers and a polyline. Finally, it saves the map as an HTML file and displays it.

1. Imports necessary libraries: folium, polyline, json, and os.
2. Sets the file name and path of the GeoJSON file (route.geojson).
3. Checks if the GeoJSON file exists in the specified path.
4. Reads the GeoJSON data from the file using json.load().
5. Extracts the coordinates of the first and last locations from the GeoJSON data.
6. Calculates the mean coordinates of the first and last locations.
7. Creates a Folium map centered around the mean coordinates.
8. Extracts the route geometry from the GeoJSON data.
9. Decodes the polyline string in the route geometry to get the coordinates.
10. Adds markers for the first and last locations on the map using folium.CircleMarker().
11. Adds the route polyline to the map using folium.PolyLine().
12. Saves the map as an HTML file named route_map.html.
13. Displays the map.

Please run the code below and see the map.

In [ ]:
import folium
import polyline
import json
import os

# Set the file name of the GeoJSON file
route_geojson = 'route.geojson'

# Set the path to the directory where route.geojson is stored
route_geojson_path = '.'

# Check if the GeoJSON file exists
if not os.path.exists(os.path.join(route_geojson_path, route_geojson)):
    print(f"Error: {route_geojson} file not found.")
    exit(1)

# Read the GeoJSON data from the file
with open(os.path.join(route_geojson_path, route_geojson)) as f:
    try:
        geojson_data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error: Failed to parse JSON data from {route_geojson}: {e}")
        exit(1)

        
# First location coordinates
lon1, lat1 = geojson_data['waypoints'][0]['location']

# Last location coordinates
lon2, lat2 = geojson_data['waypoints'][-1]['location']

# Calculate the mean coordinates
mean_lon = (lon1 + lon2) / 2
mean_lat = (lat1 + lat2) / 2

# Create a folium map centered around the route
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=7)

# Extract the route geometry
geometry = geojson_data['routes'][0]['geometry']

# Decode the polyline string to get the coordinates
coordinates = polyline.decode(geometry)

# Add markers for the first locations
folium.CircleMarker(
    location=[lat1, lon1],
    radius=6, 
    color='red',
    fill=True,
    fill_color='blue',
    ).add_to(m)

# Add markers for the final locations
folium.CircleMarker(
    location=[lat2, lon2],
    radius=6, 
    color='red',
    fill=True,
    fill_color='blue',
    ).add_to(m)

# Add the route polyline to the map
folium.PolyLine(
    locations=coordinates,
    color='blue',
    weight=5,
    opacity=0.7
).add_to(m)

# Save the map as an HTML file
m.save('images/route.html')

# Display map
m

In [ ]:
from IPython import display

# Uncomment the line below to see a screenshot of the map
# display.Image("images/route.jpg")

## Integrating Data into Visualisations:

In this section, we will read location data from a file called locations.csv, and visualise the data on the map. We also integrate a superimposed window on the map that shows additional information.

For simplicity, we use the locations we have already worked with (London and Birmingham).

The contents of locations.csv look like this:

```bash
name,region,country,latitude,longitude,location_type,conflict_date,population
London,London,GB,51.50837083467904,-0.13187578769508834,conflict_zone,1,0
Birmingham,Birmingham,GB,52.498968318134516,-1.8030022360693643,conflict_zone,1,0
```

We have added a copy of the locations.csv in the parent directory.

Please run the cell below and see the map.

In [ ]:
import pandas as pd
import requests
import os

# Set the file name of the locations.csv file
locations_csv = 'new_locations.csv'

# Check if the CSV file exists
if not os.path.exists(locations_csv):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv)

# List to keep track of invalid locations
invalid_locations = []

# Base URL for the OSRM nearest service
osrm_endpoint = 'http://localhost:5000/nearest/v1/driving/'

# Iterate through each location in the DataFrame
for index, row in df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    name = row['name']

    # Prepare API request URL for the nearest service
    request_url = f"{osrm_endpoint}{lon},{lat}"

    # Send request to OSRM
    response = requests.get(request_url)
    data = response.json()

    # Check if the response is valid and contains a way (road)
    if response.status_code == 200 and data['code'] == 'Ok' and len(data['waypoints']) > 0:
        # This means the location is valid and near a known road
        print(f"Valid location found for {name}.")
    else:
        # The location is invalid or not near a known road
        print(f"Invalid location or not found: {name}.")
        invalid_locations.append(name)

# Optional: Handle invalid locations
if invalid_locations:
    print("\nInvalid locations or locations not found:")
    for name in invalid_locations:
        print(f" - {name}")
else:
    print("\nAll locations are valid.")

In [ ]:
import pandas as pd
import folium

# Load the locations data from CSV
df = pd.read_csv('ssudan/ssudan_locations.csv')

# Assuming the DataFrame has columns 'name', 'latitude', and 'longitude'
# Create a map centered around the average latitude and longitude of the locations
center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

# Iterate through the DataFrame to add each location as a marker on the map
for index, row in df.iterrows():
    # Add markers for the final locations
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='red',
        fill=True,
        fill_color='blue',
        popup=row['name']
    ).add_to(m)


# Save the map as an HTML file
dir = "ssudan"
map_file = f'{dir}/locations_map.html'
m.save(map_file)

print(f"Map showing all locations saved as {map_file}")
m

In [ ]:
import pandas as pd
import requests
import os

# Set the file name of the locations.csv file
locations_csv = 'new_locations.csv'

# Check if the CSV file exists
if not os.path.exists(locations_csv):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv)

# List to keep track of locations without responses
locations_without_response = []

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    name1 = df.loc[i, 'name']
    lat1 = df.loc[i, 'latitude']
    lon1 = df.loc[i, 'longitude']
    
    for j in range(i + 1, len(df)):
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        # Send request to OSRM
        response = requests.get(request_url)
        
        if response.status_code != 200 or 'routes' not in response.json() or len(response.json()['routes']) == 0:
            # Either the request failed, or no route was found
            print(f"Warning: No route or response for locations {name1} and {name2}.")
            locations_without_response.append((name1, name2))
            continue  # Skip further processing and continue with the next pair

        route_data = response.json()

        # Extract route steps
        steps = route_data['routes'][0]['legs'][0]['steps']
        distance_sum = 0
        for step in steps:
            # Extract step distance and add to the distance sum
            distance = step['distance']
            distance_sum += distance
        # At this point, distance_sum contains the total distance between name1 and name2

# Optional: Display or handle locations without responses
if locations_without_response:
    print("Locations without responses or routes:")
    for loc_pair in locations_without_response:
        print(f" - {loc_pair[0]} and {loc_pair[1]}")
else:
    print("All locations returned a valid response.")

In [ ]:
import pandas as pd
import requests
import folium
import polyline

# Set the file name of the locations.csv file
locations_csv = 'new_locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the GeoJSON file exists
if not os.path.exists(os.path.join(locations_csv_path, locations_csv)):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv('{}/{}'.format(locations_csv_path, locations_csv))

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        # Send request to OSRM
        response = requests.get(request_url)
        route_data = response.json()

        # Check if response contains a route
        if 'routes' in route_data and len(route_data['routes']) > 0:
            # Extract route steps
            steps = route_data['routes'][0]['legs'][0]['steps']
            distance_sum = 0
            for step in steps:
                # Extract step instructions
                instruction = step.get('name', '') or step.get('ref', '')

                # Extract step distance and add to the distance sum
                distance = step['distance']
                distance_sum += distance

                # Append step information to the route_instructions list
                if instruction:
                    route_instructions.append((instruction, distance_sum))

            # Extract route geometry
            geometry = route_data['routes'][0]['geometry']

            # Decode polyline string to get coordinates
            coordinates = polyline.decode(geometry)

            # Create a Folium PolyLine object
            polyline_obj = folium.PolyLine(
                locations=coordinates,
                color='blue',
                weight=2,
                opacity=0.9
            )

            # Add the PolyLine to the map
            polyline_obj.add_to(m)

        # Add markers for the final locations
        folium.CircleMarker(
            location=[lat1, lon1],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        # Add markers for the final locations
        folium.CircleMarker(
            location=[lat2, lon2],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Create the side window HTML content
side_window_content = "<h2>Route Instructions:</h2>"
current_distance = 0

for instruction, distance in route_instructions:
    # Convert the distance from meters to kilometers
    distance_km = distance / 1000
    
    # Round the distance to two decimal places
    rounded_distance = round(distance_km, 2)
    
    # Format the instruction line with a colon instead of a hyphen
    formatted_instruction = f"{instruction}: {rounded_distance:.2f} km"
    
    # Add the formatted instruction to the side window content
    side_window_content += f"<p><strong>{formatted_instruction}</strong></p>"
    current_distance = distance

# Create a custom HTML element for the side window
side_window_html = f"""
<div style="position: fixed; 
            font-size: 12px;
            top: 50px; 
            right: 80px; 
            width: 30%; 
            height: 80%;
            background-color:white; 
            border:2px solid grey;
            z-index: 900;
            overflow-x: scroll;
            overflow-y: scroll;
            white-space: nowrap">
            {side_window_content}
</div>
"""

# Add the side window HTML as a custom element to the Figure
m.get_root().html.add_child(folium.Element(side_window_html))

# Save the map as an HTML file
m.save('images/route-with-instruction-map.html')

print("Map saved as route-with-instruction-map.html")

# Display map
m

In [ ]:
from IPython import display

# Please uncomment the line below to see a screenshot of the map
# display.Image("images/route-with-instructions.jpg")

## Construct More Routes

In the code below we read more locations from more-locations.csv file and construct the routes.

A copy of the more-locations.csv is added to the parent directory.

Please run the code below and see the map.

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline

# Set the file name of the locations.csv file
locations_csv = 'new_locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
if not os.path.exists(os.path.join(locations_csv_path, locations_csv)):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(os.path.join(locations_csv_path, locations_csv))

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        # Send request to OSRM
        response = requests.get(request_url)
        route_data = response.json()
        
        # Check if response contains a route
        if 'routes' in route_data and len(route_data['routes']) > 0:
            # Extract route steps
            steps = route_data['routes'][0]['legs'][0]['steps']
            distance_sum = 0
            for step in steps:
                # Extract step instructions
                instruction = step['name']

                # Extract step distance and add to the distance sum
                distance = step['distance']
                distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))
        
            # Extract route geometry
            geometry = route_data['routes'][0]['geometry']

            # Decode polyline string to get coordinates
            coordinates = polyline.decode(geometry)

            # Create a Folium PolyLine object
            polyline_obj = folium.PolyLine(
                locations=coordinates,
                color='blue',
                weight=2,
                opacity=0.9
            )

            # Add the PolyLine to the map
            polyline_obj.add_to(m)

        # Add markers for the final locations
        folium.CircleMarker(
            location=[lat1, lon1],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        # Add markers for the final locations
        folium.CircleMarker(
            location=[lat2, lon2],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)
    
# Save the map as an HTML file
m.save('images/more-route-map.html')

print("Map saved as more-route-map.html")

# Display the map
m

In [ ]:
from IPython import display

# Please uncomment the line below to see a screenshot of the map
# display.Image("images/more-route-map.jpg")

## Storing Route Data

The code below will save the route coordinates for each location pair in a CSV file called "more_routes.csv" with columns "location" and "coordinates". Each row will represent a location pair, and the "coordinates" column will contain the corresponding polyline coordinates.

In [ ]:
import os
import pandas as pd
import requests
import polyline

# Set the file name of the locations.csv file
locations_csv = 'new_locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
if not os.path.exists(os.path.join(locations_csv_path, locations_csv)):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(os.path.join(locations_csv_path, locations_csv))

# Create an empty list to store the route coordinates
route_coordinates = []

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        # Send request to OSRM
        response = requests.get(request_url)
        route_data = response.json()
        
        # Check if response contains a route
        if 'routes' in route_data and len(route_data['routes']) > 0:
            # Extract route geometry
            geometry = route_data['routes'][0]['geometry']
            
            # Decode polyline string to get coordinates
            coordinates = polyline.decode(geometry)
            
            # Store the coordinates as a dictionary
            route_coordinates.append({'location': f"{name1} - {name2}", 'coordinates': coordinates})

# Convert the list of dictionaries to a DataFrame
route_coordinates_df = pd.DataFrame(route_coordinates)

# Save the route coordinates DataFrame to a CSV file
route_coordinates_df.to_csv('new_routes.csv', index=False)

print("Route coordinates saved as more-routes.csv")

Visualise the CSV data in a map by running the script below:

In [ ]:
import os
import pandas as pd
import folium
import ast

# Set the file name of the routes.csv file
routes_csv = 'new_routes.csv'

# Set the path to the directory where routes.csv is stored
routes_csv_path = '.'

# Check if the CSV file exists
if not os.path.exists(os.path.join(routes_csv_path, routes_csv)):
    print(f"Error: {routes_csv} file not found.")
    exit(1)

# Read the route coordinates from CSV
df = pd.read_csv(os.path.join(routes_csv_path, routes_csv))

# Create a map centered around the first location
start_lat, start_lon = ast.literal_eval(df['coordinates'][0])[0]
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Iterate through each route
for _, row in df.iterrows():
    # Extract route details
    location = row['location']
    coordinates = ast.literal_eval(row['coordinates'])

    # Create a Folium PolyLine object
    polyline_obj = folium.PolyLine(
        locations=coordinates,
        color='blue',
        weight=1.0,
        opacity=0.9
    )

    # Add the PolyLine to the map
    polyline_obj.add_to(m)

    # Add markers for the first and last coordinate
    start_coordinate = coordinates[0]
    end_coordinate = coordinates[-1]

    start_lat, start_lon = start_coordinate
    end_lat, end_lon = end_coordinate

    folium.CircleMarker(
        location=[start_lat, start_lon],
        radius=6,
        color='red',
        fill=True,
        fill_color='blue',
        popup=f"Start: {location}"
    ).add_to(m)

    folium.CircleMarker(
        location=[end_lat, end_lon],
        radius=6,
        color='green',
        fill=True,
        fill_color='blue',
        popup=f"End: {location}"
    ).add_to(m)

# Save the map as an HTML file
m.save('more-route-map.html')

print("Map saved as more-route-map.html")

# Display the map
m

Please feel free to modify and experiment with the code.